### Начинаем учебные соревнования 

13 марта 2023 года, https://www.kaggle.com/

1) Посмотреть файлы в архиве zip (имя и размер)

In [7]:
from zipfile import ZipFile

ZipFile('/Users/tony_builder/Documents/GitHub/kaggle/datas/store-sales-time-series-forecasting.zip').infolist()

[<ZipInfo filename='holidays_events.csv' compress_type=deflate file_size=22309 compress_size=2232>,
 <ZipInfo filename='oil.csv' compress_type=deflate file_size=20580 compress_size=5822>,
 <ZipInfo filename='sample_submission.csv' compress_type=deflate file_size=342153 compress_size=65352>,
 <ZipInfo filename='stores.csv' compress_type=deflate file_size=1387 compress_size=490>,
 <ZipInfo filename='test.csv' compress_type=deflate file_size=1022269 compress_size=152503>,
 <ZipInfo filename='train.csv' compress_type=deflate file_size=121800373 compress_size=21820633>,
 <ZipInfo filename='transactions.csv' compress_type=deflate file_size=1552637 compress_size=368309>]

2. Считать конкретный файл в zip архиве и прочитать его (например через pandas.read)

In [10]:
import pandas as pd

with ZipFile('/Users/tony_builder/Documents/GitHub/kaggle/datas/store-sales-time-series-forecasting.zip', 'r') \
    as zip_file:
    holiday_df = pd.read_csv(zip_file.open('holidays_events.csv'))
    oil_df = pd.read_csv(zip_file.open('oil.csv'))
    sample_submission_df = pd.read_csv(zip_file.open('sample_submission.csv'))
    test = pd.read_csv(zip_file.open('test.csv'))
    train = pd.read_csv(zip_file.open('train.csv'))
    transactions = pd.read_csv(zip_file.open('transactions.csv'))



In [17]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8
